In [16]:
!pip install earthengine-api geemap

import ee
import geemap
import pandas as pd

# --- 2. Authentication and Initialization ---
try:
    ee.Authenticate()
    ee.Initialize(project='phonic-ceremony-474418-r9') # <-- PASTE YOUR PROJECT ID HERE IF NEEDED
    print("\n✅ Earth Engine Initialized Successfully.")
except Exception as e:
    print(f"❌ Error during initialization: {e}")


✅ Earth Engine Initialized Successfully.


In [22]:
try:
    admin_boundaries = ee.FeatureCollection('FAO/GAUL/2015/level2')
    delhi_aoi = admin_boundaries.filter(ee.Filter.eq('ADM1_NAME', 'Delhi'))
    print("Boundary file loaded successfully for the entire Delhi NCT.")
except Exception as e:
    print(f"❌ Error loading boundary file: {e}")


# --- 4. Process Satellite Imagery ---
try:
    years_to_process = [2011, 2013, 2015, 2017, 2019]
    ndvi_layers = {}

    def mask_sr_clouds(image):
        qa = image.select('QA_PIXEL')
        cloud_shadow_bit_mask = (1 << 3)
        clouds_bit_mask = (1 << 5)
        mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(qa.bitwiseAnd(clouds_bit_mask).eq(0))
        return image.updateMask(mask)

    def calculate_ndvi_ls57(image):
        return image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')

    def calculate_ndvi_ls89(image):
        return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

    for year in years_to_process:
        try:
            print(f"\n--- Processing year {year} ---")
            start_date = f'{year}-01-01'
            end_date = f'{year}-12-31'

            if year < 2013:
                collection_id = 'LANDSAT/LE07/C02/T1_L2'
                ndvi_function = calculate_ndvi_ls57
            else:
                collection_id = 'LANDSAT/LC08/C02/T1_L2'
                ndvi_function = calculate_ndvi_ls89

            collection = ee.ImageCollection(collection_id).filterBounds(delhi_aoi).filterDate(start_date, end_date)

            if collection.size().getInfo() == 0:
                raise ee.EEException(f"No satellite images found for {year}.")

            ndvi_image = collection.map(mask_sr_clouds).map(ndvi_function).median().clip(delhi_aoi)

            ndvi_layers[year] = ndvi_image
            print(f"✅ Successfully created NDVI layer for {year}.")

        except Exception as e:
            print(f"⚠️ Could not process year {year}: {e}. Skipping.")
            continue

    # --- 5. Calculate and Display Key Indicators ---
    if ndvi_layers:
        print("\nCalculating yearly NDVI statistics...")
        ndvi_stats = []
        reducer = ee.Reducer.mean().combine(ee.Reducer.stdDev(), '', True).combine(ee.Reducer.minMax(), '', True)

        for year, layer in ndvi_layers.items():
            stats = layer.reduceRegion(reducer=reducer, geometry=delhi_aoi.geometry(), scale=30, maxPixels=1e9)
            stats_dict = stats.getInfo()
            ndvi_stats.append({
                'Year': year, 'Mean_NDVI': stats_dict.get('NDVI_mean'), 'Std_Dev_NDVI': stats_dict.get('NDVI_stdDev'),
                'Min_NDVI': stats_dict.get('NDVI_min'), 'Max_NDVI': stats_dict.get('NDVI_max')
            })

        summary_df = pd.DataFrame(ndvi_stats).round(4)
        print("\n" + "="*60)
        print("           Table 1: Key NDVI Indicators for Delhi NCT")
        print("="*60)
        print(summary_df.to_string(index=False))

    # --- 6. Visualize Maps and Calculate Change Indicators ---
    if len(ndvi_layers) >= 2:
        print("\nGenerating interactive map and change statistics...")
        Map = geemap.Map()
        Map.centerObject(delhi_aoi, 10)

        ndvi_vis_params = {'min': -0.2, 'max': 0.8, 'palette': ['red', 'yellow', 'green']}
        diff_vis_params = {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'green']}

        for year, layer in ndvi_layers.items():
            Map.addLayer(layer, ndvi_vis_params, f'NDVI {year}', False)

        available_years = sorted(ndvi_layers.keys())
        change_stats = []

        # Calculate and add pairwise change maps and stats
        for i in range(len(available_years) - 1):
            start_year, end_year = available_years[i], available_years[i+1]
            pairwise_diff = ndvi_layers[end_year].subtract(ndvi_layers[start_year])
            Map.addLayer(pairwise_diff, diff_vis_params, f'NDVI Change ({start_year}-{end_year})', False)

            stats = pairwise_diff.reduceRegion(reducer=reducer, geometry=delhi_aoi.geometry(), scale=30, maxPixels=1e9)
            stats_dict = stats.getInfo()
            change_stats.append({
                'Period': f'{start_year}-{end_year}', 'Mean_Change_NDVI': stats_dict.get('NDVI_mean'),
                'Min_Change': stats_dict.get('NDVI_min'), 'Max_Change': stats_dict.get('NDVI_max')
            })

        # Calculate and add overall change map and stats
        first_year, last_year = available_years[0], available_years[-1]
        overall_difference = ndvi_layers[last_year].subtract(ndvi_layers[first_year])
        Map.addLayer(overall_difference, diff_vis_params, f'NDVI Change ({first_year}-{last_year})', True)
        stats = overall_difference.reduceRegion(reducer=reducer, geometry=delhi_aoi.geometry(), scale=30, maxPixels=1e9)
        stats_dict = stats.getInfo()
        change_stats.append({
            'Period': f'{first_year}-{last_year}', 'Mean_Change_NDVI': stats_dict.get('NDVI_mean'),
            'Min_Change': stats_dict.get('NDVI_min'), 'Max_Change': stats_dict.get('NDVI_max')
        })

        # Display the change statistics table
        change_df = pd.DataFrame(change_stats).round(4)
        print("\n" + "="*60)
        print("          Table 2: Key NDVI Change Indicators")
        print("="*60)
        print(change_df.to_string(index=False))

        # Add colorbar, save, and display map
        Map.add_colorbar(ndvi_vis_params, label="NDVI (Vegetation Index)")
        output_html = 'ndvi_interactive_map_final.html'
        Map.to_html(output_html)
        print(f"\n✅ Final interactive map saved to '{output_html}'.")
        display(Map)
    else:
        print("\nNot enough data to create change maps or statistics.")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


Boundary file loaded successfully for the entire Delhi NCT.

--- Processing year 2011 ---
✅ Successfully created NDVI layer for 2011.

--- Processing year 2013 ---
✅ Successfully created NDVI layer for 2013.

--- Processing year 2015 ---
✅ Successfully created NDVI layer for 2015.

--- Processing year 2017 ---
✅ Successfully created NDVI layer for 2017.

--- Processing year 2019 ---
✅ Successfully created NDVI layer for 2019.

Calculating yearly NDVI statistics...

           Table 1: Key NDVI Indicators for Delhi NCT
 Year  Mean_NDVI  Std_Dev_NDVI  Min_NDVI  Max_NDVI
 2011     0.1081        0.0471   -0.0492    0.3566
 2013     0.1252        0.0507   -0.0353    0.3949
 2015     0.1368        0.0546   -0.0185    0.3735
 2017     0.1344        0.0546   -0.0246    0.3807
 2019     0.1361        0.0557   -0.0254    0.3987

Generating interactive map and change statistics...

          Table 2: Key NDVI Change Indicators
   Period  Mean_Change_NDVI  Min_Change  Max_Change
2011-2013         

Map(center=[28.646455931994893, 77.10900424302301], controls=(WidgetControl(options=['position', 'transparent_…